# Accessing OpenAI through Vocareum API keys

In [34]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-8957811371266772085699678d66e1475c97.62448043"

In [37]:
from openai import OpenAI

client = OpenAI(
    api_key="voc-8957811371266772085699678d66e1475c97.62448043",
    base_url="https://openai.vocareum.com/v1"
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "Talk like a pirate."},
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        },
    ],
)

print(completion.choices[0].message.content)

Arrr matey! If ye be wantin' to check if a Python object be an instance of a class, ye can use the good ol' `isinstance()` function. Here’s how ye do it, savvy?

```python
if isinstance(your_object, YourClass):
    print("Ye object be an instance of the class!")
else:
    print("That not be the class ye be lookin' for!")
```

Replace `your_object` with the treasure ye be searchin’, and `YourClass` with the class ye be checkin’ against. This’ll tell ye if yer object matches the class or not. Yarrr!


In [28]:
from openai import OpenAI
client = OpenAI(
    base_url = openai.api_base,
    api_key = openai.api_key
)

In [17]:
def print_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)


# Extracting data from Wikipedia

## Getting a list of all pages from a Wikipedia category

In [4]:
import wikipediaapi
import openai
import pandas as pd
import json
import os

In [5]:
def fetch_wikipedia_page(page_title):
    page = wiki_wiki.page(page_title)
    if page.exists():
        return page.text
    else:
        return None

In [6]:
wiki_wiki = wikipediaapi.Wikipedia('CustomChatbot (gabrielgoncalvesbr@gmail.com)', 'en')

In [7]:
if os.path.exists("nba_playoffs_wikipedia.json"):
    # Read the JSON file and load it into the dictionary
    with open("nba_playoffs_wikipedia.json", "r", encoding="utf-8") as json_file:
        dict_nba_playoffs_pages = json.load(json_file)
else:
    dict_nba_playoffs_pages = {}
    
    cat = wiki_wiki.page("Category:NBA playoffs")
    for title in cat.categorymembers:
        print(title)
        page_result = fetch_wikipedia_page(title)
        if page_result:
            dict_nba_playoffs_pages[title] = page_result

    with open("nba_playoffs_wikipedia.json", "w", encoding="utf-8") as json_file:
        json.dump(dict_nba_playoffs_pages, json_file, ensure_ascii=False, indent=4)


# Preparing the dataset

In [15]:
df_playoffs = pd.DataFrame.from_dict(
    dict_nba_playoffs_pages, 
    orient='index', 
    columns=['text']#, 'text']
).reset_index().rename(columns={'index': 'title'})
df_playoffs.head()

,title,text
0,NBA playoffs,The NBA playoffs is the annual elimination tou...
1,1950 NBA playoffs,The 1950 NBA playoffs was the postseason tourn...
2,1951 NBA playoffs,The 1951 NBA playoffs was the postseason tourn...
3,1952 NBA playoffs,The 1952 NBA playoffs was the postseason tourn...
4,1953 NBA playoffs,The 1953 NBA playoffs was the postseason tourn...


In [16]:
df_playoffs[df_playoffs['title'].str.contains("2023")]

,title,text
74,2023 NBA playoffs,The 2023 NBA playoffs was the postseason tourn...


In [10]:
df_playoffs.columns

RangeIndex(start=0, stop=1, step=1)

# Test Chatbot

In [29]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-8957811371266772085699678d66e1475c97.62448043"

In [28]:
from openai import OpenAI
client = OpenAI(
    base_url = openai.api_base,
    api_key = openai.api_key
)

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="voc-8957811371266772085699678d66e1475c97.62448043",
    base_url="https://openai.vocareum.com/v1"
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "Talk like a pirate."},
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        },
    ],
)

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key="voc-8957811371266772085699678d66e1475c97.62448043",
    base_url="https://openai.vocareum.com/v1"
)

def chat_with_openai(messages):
    """
    Sends a conversation to OpenAI and gets a response using the updated API.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Replace with "gpt-4" or "gpt-3.5-turbo" as needed
            messages=messages
        )
        # Access the content of the response using dot notation
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def naive_OpenAI_chatbot():
    """
    Starts the chatbot interaction.
    """
    print("👋 Welcome to the NBA Playoffs Chatbot!")
    
    # Initialize conversation history
    conversation = [
        {"role": "system", "content": "You are a helpful assistant knowledgeable about NBA playoffs."}
    ]
    
    # Ask the first question
    year = input("Bot: Which NBA playoff year do you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about the NBA playoffs in {year}."})
    
    # Ask the second question
    info_type = input("Bot: What's the information you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about {info_type} in the {year} NBA playoffs."})
    
    # Generate a response
    print("Bot: Let me gather some information for you...\n")
    
    assistant_prompt = f"Generate a detailed paragraph about {info_type} in the {year} NBA playoffs."
    conversation.append({"role": "assistant", "content": assistant_prompt})
    
    response = chat_with_openai(conversation)
    
    if response:
        # Display the generated paragraph
        print(f"Bot: Here's what I found:\n{response}\n")
        
        # Confirm with the user
        is_correct = input("Bot: Is this what you were searching for? (yes/no)\nYou: ").strip().lower()
        
        if is_correct == 'yes':
            print("Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.")
        else:
            print("Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?")
            start_chatbot()  # Restart if needed
    else:
        print("Bot: I couldn't retrieve any information. Please try again later.")


In [31]:
start_chatbot()

👋 Welcome to the NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  2022
Bot: What's the information you want to know about?
You:  Which team won and what's who was the finals MVP?


Bot: Let me gather some information for you...



APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


# Exercise Chatbot Memory

In [22]:
import os
import json
import openai

In [23]:
# decoding parameters
TEMPERATURE = 0.0
MAX_TOKENS = 5000
TOP_P = 1.0

In [24]:
SYSTEM_PROMPT = """You expert at games of chance.
End every response with double exclamation points!!"""

USER_NAME = "User"
AI_NAME = "AI Assistant"
NEW_INTERACTION_DELIMITER = "\n\n"

# Creating a chat bot with memory
Using the basic `openai.Completion` API to understand where the memory "lives" in the prompt.

# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

### Question 2